In [ ]:
from multiprocessing import freeze_support
import pandas as pd
from datasets import load_dataset
from simpletransformers.classification import ClassificationArgs, ClassificationModel
import numpy as np
from sklearn.metrics import classification_report, f1_score


In [ ]:
tomatoes = load_dataset("rotten_tomatoes")
train_df = pd.DataFrame(tomatoes["train"])
test_df = pd.DataFrame(tomatoes["test"])

In [ ]:
model_args = ClassificationArgs()
model_args.train_custom_parameters_only = True
model_args.custom_parameter_groups = [
        {"params": ["classifier.weight"], "lr": 1e-3},
        {"params": ["classifier.bias"], "lr": 1e-3, "weight_decay": 0.0},
]

model = ClassificationModel(model_type="bert", model_name="bert-base-cased", args=model_args, use_cuda=False)

model.train_model(train_df)

In [ ]:
result, model_outputs, wrong_predictions = model.eval_model(test_df, f1=f1_score)
y_pred = np.argmax(model_outputs, axis=1)
print(classification_report(test_df.label, y_pred))